In [1]:
from pyroutelib3 import Router, Datastore # Import the router
router = Router("cycle", "C:/Users/pfrod/OneDrive/Documents/1A mines de paris/MIG/carte_test.osm") # Initialise it


In [2]:
start = router.findNode(43.7006971, 7.2587751) # Find start and end nodes
end = router.findNode(43.700561, 7.2599291)

In [3]:
status, route = router.doRoute(start, end) # Find the route - a list of OSM nodes


In [4]:
status

'success'

In [5]:
route

[564661114,
 3667118231,
 273485738,
 1556416769,
 1556416745,
 273485740,
 1701090176,
 311258859,
 24343181,
 311258915,
 311258917]

In [6]:
list(map(router.nodeLatLon, route))

[(43.7006972, 7.2587752),
 (43.7006331, 7.258767),
 (43.700484, 7.2587479),
 (43.7004658, 7.2587252),
 (43.7004336, 7.258622),
 (43.7004036, 7.2586002),
 (43.6996616, 7.2584458),
 (43.6995896, 7.2584263),
 (43.6995951, 7.259249),
 (43.6995986, 7.2597091),
 (43.700563, 7.2599293)]

In [57]:
surface = 1  #récupérer la carte ensoleillement et altitude avec les points dessus 
(pas,pastemp) = (7000000,1)
(xo,yo,to) = (3,2,0)
vitesse = 20/3.6
import numpy as np
puiscarac = 140 #typiquement la puissance moyenne restituée par un panneau sur un triporteur
cartalt = np.random.randint(50, size=(80,80))
cartensol =np.random.randint(50, size=(80,80, 5))
from pyproj import Transformer
transformer_to_lamb= Transformer.from_crs("EPSG:4326", "EPSG:2154", always_xy=True)
transformer_to_lat_long=Transformer.from_crs( "EPSG:2154","EPSG:4326", always_xy=True)
L=[(43.7006972, 7.2587752),
 (43.7006331, 7.258767),
 (43.700484, 7.2587479),
 (43.7004658, 7.2587252),
 (43.7004336, 7.258622),
 (43.7004036, 7.2586002),
 (43.6996616, 7.2584458),
 (43.6995896, 7.2584263),
 (43.6995951, 7.259249),
 (43.6995986, 7.2597091),
 (43.700563, 7.2599293)]
G=[transformer_to_lamb.transform(L[i][0],L[i][1]) for i in range(len(L))]
class Point :
    def __init__(self, lat, lon) :
        self.latitude = lat
        self.longitude = lon
        self.id = router.findNode(lat, lon)
        self.x = transformer_to_lamb.transform(lat,lon)[0]
        self.y = transformer_to_lamb.transform(lat,lon)[1]
        self.alti = cartalt[round((self.x-xo) / pas)][round((self.y-yo) / pas)]
    
    def __repr__(self) :
        return (f"point de latitude {self.latitude}, de longitude {self.longitude}, d'altitude {self.alti}, x={self.x}, y={self.y} ")


def ensol(point, t) :
    pix_x = round((point.x-xo) / pas)
    pix_y = round((point.y-yo) / pas)
    pix_t = round((t-to) / pastemp)
    return cartensol[pix_x, pix_y, pix_t] * surface


def distroute(point1, point2):
    start = point1.id # Find start and end nodes
    end = point2.id
    status, route = router.doRoute(start, end)
    l = 0
    for i in range(len(route)-1):
        point1 = Point(router.rnodes[route[i]][0],router.rnodes[route[i]][1]) 
        point2 = Point(router.rnodes[route[i+1]][0],router.rnodes[route[i+1]][1])
        l += router.distance((point1.latitude, point1.longitude),(point2.latitude, point2.longitude))
    return l * 1000


def enersol(point1, point2, t):
    start = point1.id # Find start and end nodes
    end = point2.id
    status, route = router.doRoute(start, end)
    l=0
    for i in range(len(route)-1) :
        point1 = Point(router.rnodes[route[i]][0],router.rnodes[route[i]][1]) 
        point2 = Point(router.rnodes[route[i+1]][0],router.rnodes[route[i+1]][1])
        l += router.distance((point1.latitude, point1.longitude),(point2.latitude, point2.longitude)) * ensol(point1, t) * vitesse * 1000
    return l 


def deniv(point1, point2) :
    start = point1.id # Find start and end nodes
    end = point2.id
    status, route = router.doRoute(start, end)
    l = 0
    for i in range(len(route)-1):
        point1 = Point(router.rnodes[route[i]][0],router.rnodes[route[i]][1]) 
        point2 = Point(router.rnodes[route[i+1]][0],router.rnodes[route[i+1]][1])
        l += max(0,point2.alti-point1.alti)
    return l


def graph(nodeslist, t, a, b, c) :#nodeslist est la liste des (x,y) des points de livraison 
    if not(( 0 <= a <= 1 ) and ( 0 <= b <= 1 ) and ( 0 <= c <= 1 )):
        raise ValueError("Les paramètres sont entre 0 et 1")
    for i in range(len(nodeslist)) :
        nodeslist[i] = Point(transformer_to_lat_long.transform(nodeslist[i][0],nodeslist[i][1])[0],transformer_to_lat_long.transform(nodeslist[i][0],nodeslist[i][1])[1])
    def poids(point1,point2,t):
        return a * distroute(point1, point2) + b * 100 * deniv(point1, point2) + c * vitesse / puiscarac * enersol(point1, point2, t)
    graphe = {}
    for s in nodeslist :
        graphe[s] = {}
        for j in nodeslist : 
            if j!=s:
                graphe[s][j] = poids(s, j, t)
    return graphe
graph(G, 1, 0.5, 0.2, 0.7)
#print(router.findNode(transformer_to_lat_long.transform(2.8,3.3)[0], transformer_to_lat_long.transform(2.8,3.3)[1]))
#print(router.findNode(transformer_to_lat_long.transform(25,1.1)[0], transformer_to_lat_long.transform(25,1.1)[1]))

{point de latitude 43.7006972, de longitude 7.258775199999395, d'altitude 17, x=5986987.626214701, y=3323340.3211777466 : {point de latitude 43.70063309999998, de longitude 7.258766999999393, d'altitude 17, x=5986980.600304716, y=3323335.0587997627 : 15.72995340014509,
  point de latitude 43.70048400000001, de longitude 7.258747899999396, d'altitude 17, x=5986964.259413862, y=3323322.8151289853 : 52.319089129213424,
  point de latitude 43.7004658, de longitude 7.258725199999401, d'altitude 17, x=5986963.630493745, y=3323318.909852059 : 58.30735606632868,
  point de latitude 43.7004336, de longitude 7.258621999999392, d'altitude 17, x=5986966.744619365, y=3323304.53950461 : 78.16396438658691,
  point de latitude 43.700403599999994, de longitude 7.258600199999402, d'altitude 17, x=5986964.660746019, y=3323299.950669517 : 86.44463923818674,
  point de latitude 43.6996616, de longitude 7.2584457999994, d'altitude 17, x=5986887.318605102, y=3323231.99580086 : 269.7957545683632,
  point de l

In [8]:
router.rnodes[311258859]
print(router.routing[273485738])
print(distroute(Point(router.rnodes[311258859][0],router.rnodes[311258859][1]),Point(router.rnodes[564661114][0],router.rnodes[564661114][1])))
point1=Point(2.0,3.2)
print(point1)

{1556416769: 2.5}


TypeError: list indices must be integers or slices, not tuple

In [ ]:
from pyproj import Transformer
transformer_to_lamb= Transformer.from_crs("EPSG:4326", "EPSG:2154", always_xy=True)
transformer_to_lat_long=Transformer.from_crs( "EPSG:2154","EPSG:4326", always_xy=True)

#La syntaxe à utiliser est :
transformer_to_lamb.transform(7.26528, 43.70168)